# 🚀 RAFT Fine-tuning on MSRS Story-QA

Complete pipeline for fine-tuning Qwen3-4B-Instruct with RAFT methodology on Google Colab T4 GPU.

## 📋 What This Notebook Does

1. **Setup Environment** - Install dependencies and clone repository
2. **Load Data** - Download MSRS Story-QA dataset
3. **Build Index** - Create vector search index
4. **Generate RAFT Dataset** - Create training data with CoT and citations
5. **Train Model** - Fine-tune with Unsloth QLoRA
6. **Evaluate** - Test model performance

## ⚙️ Requirements

- **GPU**: T4 (15GB VRAM)
- **RAM**: High-RAM runtime recommended
- **Time**: ~3-4 hours for 50-100 training examples
- **API Key**: OpenAI API key for CoT generation

## 🎯 Quick Start

1. Enable GPU: Runtime → Change runtime type → T4 GPU
2. Run all cells in order
3. Enter your OpenAI API key when prompted
4. Wait for training to complete

---

## 🔧 Step 1: Setup Environment

Install all required packages and clone the repository.

In [ ]:
%%capture
# Install PyTorch with CUDA support
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
%%capture
# Clone the repository
!git clone https://github.com/limcheekin/MSRS-RAFT.git
%cd MSRS-RAFT

In [ ]:
%%capture
# Install all dependencies
!pip install -r requirements.txt

# Upgrade Unsloth to latest
!pip install --upgrade --force-reinstall --no-cache-dir unsloth unsloth_zoo

# Download NLTK data
import nltk
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)

In [ ]:
# Verify installation
print("🔍 Verifying installation...\n")
!python test_installation.py

## 🔑 Step 2: Configure API Keys and Settings

Set up your OpenAI API key and configure training parameters.

In [ ]:
import os
from getpass import getpass

# Get OpenAI API key
print("📝 Enter your OpenAI API key (required for RAFT dataset generation):")
openai_key = getpass("OpenAI API Key: ")
os.environ['OPENAI_API_KEY'] = openai_key

print("\n✅ API key configured!")

In [ ]:
# Configure training parameters for T4 GPU
from raft_config import RAFTConfig, ModelConfig, TrainingConfig, RAFTDataConfig

# Create custom config optimized for T4
config = RAFTConfig()

# Model settings (optimized for T4 15GB)
config.model.max_seq_length = 2048  # Reduced for T4
config.model.lora_r = 16  # Smaller LoRA rank
config.model.lora_alpha = 32
config.model.load_in_4bit = True

# Training settings (optimized for T4)
config.training.num_train_epochs = 2  # Fewer epochs for demo
config.training.per_device_train_batch_size = 1  # Small batch for T4
config.training.gradient_accumulation_steps = 8  # Effective batch size = 8
config.training.learning_rate = 2e-4
config.training.max_new_tokens = 512  # Reduced for T4
config.training.logging_steps = 10
config.training.eval_steps = 50
config.training.save_steps = 100

# RAFT settings
config.raft_data.oracle_percentage = 0.8
config.raft_data.num_distractors = 3  # Fewer distractors
config.raft_data.chunk_size = 1000  # Smaller chunks

# System settings
config.system.project_name = "raft-colab"
config.system.use_wandb = False  # Disable W&B for simplicity

# Save config
config.to_yaml("colab_config.yaml")

print("⚙️ Configuration for T4 GPU:")
print(f"  Max Sequence Length: {config.model.max_seq_length}")
print(f"  LoRA Rank: {config.model.lora_r}")
print(f"  Batch Size: {config.training.per_device_train_batch_size}")
print(f"  Gradient Accumulation: {config.training.gradient_accumulation_steps}")
print(f"  Effective Batch Size: {config.training.per_device_train_batch_size * config.training.gradient_accumulation_steps}")
print(f"  Epochs: {config.training.num_train_epochs}")
print(f"\n✅ Configuration saved to colab_config.yaml")

## 📊 Step 3: Load and Explore Data

Load the MSRS Story-QA dataset and explore its structure.

In [ ]:
from raft_data_loader import MSRSDataLoader
import json

print("📚 Loading MSRS Story-QA dataset...\n")

# Initialize loader
loader = MSRSDataLoader(
    dataset_name="yale-nlp/MSRS",
    dataset_config="story-qa",
    cache_dir="./cache"
)

# Load dataset
dataset = loader.load_dataset()

# Parse examples
train_examples = loader.parse_examples(split="train", max_examples=10)
dev_examples = loader.parse_examples(split="dev", max_examples=5)

# Load corpus
print("\n📖 Loading story corpus...")
try:
    corpus = loader.load_corpus()
    print(f"✅ Loaded {len(corpus)} chapters")
except Exception as e:
    print(f"⚠️ Could not load corpus from HuggingFace: {str(e)}")
    print("Creating minimal corpus from examples...")
    # This will work even without full corpus

# Display statistics
stats = loader.get_statistics()
print("\n📊 Dataset Statistics:")
print(json.dumps(stats, indent=2))

# Show example
print("\n📝 Example Query:")
print(f"  Query: {train_examples[0].query[:100]}...")
print(f"  Gold Documents: {train_examples[0].gold_documents[:3]}")
print(f"  Number of Answers: {len(train_examples[0].answers)}")

## 🔍 Step 4: Build Retrieval Index

Create vector search index for document retrieval.

In [ ]:
from raft_retrieval import RetrievalSystem
from raft_data_loader import Chapter
import torch

print("🔍 Building retrieval index...\n")

# Check if corpus is available
if not loader._corpus:
    print("⚠️ No corpus available. Creating mock corpus for demonstration...")
    # Create minimal corpus from examples
    loader._corpus = {}
    for i, example in enumerate(train_examples + dev_examples):
        for doc_id in example.gold_documents:
            if doc_id not in loader._corpus:
                loader._corpus[doc_id] = Chapter(
                    doc_id=doc_id,
                    story_id=doc_id.rsplit('_', 1)[0] if '_' in doc_id else doc_id,
                    chapter_index=0,
                    text=f"Sample chapter content for {doc_id}. " + example.query,
                    token_length=50
                )

# Initialize retrieval system
retrieval_system = RetrievalSystem(
    embedding_model="BAAI/bge-small-en-v1.5",  # Smaller model for T4
    reranker_model=None,  # Disable reranker to save memory
    chunk_size=config.raft_data.chunk_size,
    chunk_overlap=config.raft_data.chunk_overlap,
    device="cuda" if torch.cuda.is_available() else "cpu"
)

# Prepare documents
documents = {
    doc_id: chapter.text
    for doc_id, chapter in loader._corpus.items()
}

print(f"📚 Indexing {len(documents)} documents...")

# Build index
retrieval_system.build_index(
    documents,
    batch_size=16,  # Smaller batch for T4
    save_path="./indices/colab_index"
)

print("\n✅ Index built successfully!")

# Test retrieval
print("\n🔎 Testing retrieval...")
test_query = train_examples[0].query
results = retrieval_system.retrieve(test_query, top_k=3)

print(f"Query: {test_query[:80]}...")
print(f"\nTop 3 results:")
for i, result in enumerate(results, 1):
    print(f"  {i}. {result.doc_id} (score: {result.score:.4f})")
    print(f"     {result.text[:100]}...")

## 🏗️ Step 5: Generate RAFT Training Dataset

Create RAFT training examples with Chain-of-Thought reasoning and citations.

**Note**: This step uses OpenAI API and will incur costs (~$0.01-0.03 per example).

In [ ]:
from raft_dataset_builder import CoTGenerator, RAFTDatasetBuilder, prepare_examples_from_loader
import time

print("🏗️ Generating RAFT training dataset...\n")

# Set number of examples (adjust based on your budget)
NUM_TRAIN_EXAMPLES = 20  # Start small for testing
NUM_DEV_EXAMPLES = 5

print(f"📊 Configuration:")
print(f"  Training examples: {NUM_TRAIN_EXAMPLES}")
print(f"  Dev examples: {NUM_DEV_EXAMPLES}")
print(f"  Estimated cost: ${(NUM_TRAIN_EXAMPLES + NUM_DEV_EXAMPLES) * 0.02:.2f}")
print(f"  Estimated time: {(NUM_TRAIN_EXAMPLES + NUM_DEV_EXAMPLES) * 2} minutes\n")

# Ask for confirmation
print("⏳ Starting in 5 seconds... (interrupt if you want to change settings)")
time.sleep(5)

# Initialize CoT generator
cot_generator = CoTGenerator(
    model="gpt-4-turbo-preview",
    temperature=0.2,
    max_tokens=1500,
    api_key=os.environ.get('OPENAI_API_KEY')
)

# Initialize RAFT builder
raft_builder = RAFTDatasetBuilder(
    retrieval_system=retrieval_system,
    cot_generator=cot_generator,
    oracle_percentage=config.raft_data.oracle_percentage,
    num_distractors=config.raft_data.num_distractors,
    distractor_pool_size=10,
    max_quote_length=300,
    min_quotes=1,
    max_quotes=3
)

# Prepare examples
print("\n📝 Preparing examples from data loader...")
train_prep = prepare_examples_from_loader(loader, split="train")
dev_prep = prepare_examples_from_loader(loader, split="dev")

# Build training dataset
print(f"\n🔨 Building RAFT training dataset ({NUM_TRAIN_EXAMPLES} examples)...")
print("This will take a while. Progress will be shown below.\n")

train_raft_examples = raft_builder.build_dataset(
    train_prep[:NUM_TRAIN_EXAMPLES],
    output_path="./data/raft_train_colab.jsonl"
)

# Build dev dataset
print(f"\n🔨 Building RAFT dev dataset ({NUM_DEV_EXAMPLES} examples)...\n")

dev_raft_examples = raft_builder.build_dataset(
    dev_prep[:NUM_DEV_EXAMPLES],
    output_path="./data/raft_dev_colab.jsonl"
)

# Show statistics
train_stats = raft_builder.get_statistics(train_raft_examples)
print("\n📊 RAFT Dataset Statistics:")
print(json.dumps(train_stats, indent=2))

# Show example
if train_raft_examples:
    example = train_raft_examples[0]
    print("\n📝 Example RAFT Training Data:")
    print(f"  Query: {example.query[:100]}...")
    print(f"  Contexts: {len(example.contexts)}")
    print(f"  Oracle IDs: {example.oracle_ids}")
    print(f"  Distractor IDs: {example.distractor_ids}")
    print(f"  Reasoning (first 200 chars): {example.reasoning[:200]}...")
    print(f"  Answer: {example.answer[:100]}...")

print("\n✅ RAFT dataset generation complete!")

## 🎯 Step 6: Train Model with Unsloth

Fine-tune Qwen3-4B-Instruct using QLoRA with Unsloth.

In [ ]:
from raft_trainer import RAFTTrainer, LoggingCallback
import torch

print("🎯 Starting model training...\n")

# Check GPU
print(f"🖥️ Device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")
if torch.cuda.is_available():
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB\n")

# Initialize trainer
trainer = RAFTTrainer(config)

# Load model
print("📦 Loading Qwen3-4B-Instruct with QLoRA...")
trainer.load_model()

print("\n✅ Model loaded successfully!")

# Load datasets
print("\n📚 Loading training datasets...")
train_dataset, eval_dataset = trainer.load_dataset(
    train_path="./data/raft_train_colab.jsonl",
    eval_path="./data/raft_dev_colab.jsonl"
)

print(f"  Training examples: {len(train_dataset)}")
print(f"  Evaluation examples: {len(eval_dataset) if eval_dataset else 0}")

# Setup callbacks
callbacks = [LoggingCallback("./logs/training_colab.jsonl")]

# Show training info
total_steps = (len(train_dataset) // (config.training.per_device_train_batch_size * config.training.gradient_accumulation_steps)) * config.training.num_train_epochs
print(f"\n📊 Training Configuration:")
print(f"  Total steps: ~{total_steps}")
print(f"  Logging every {config.training.logging_steps} steps")
print(f"  Evaluation every {config.training.eval_steps} steps")
print(f"  Saving every {config.training.save_steps} steps")

print("\n🚀 Starting training...")
print("This will take 1-2 hours. Progress will be shown below.\n")
print("="*70)

# Train
train_result = trainer.train(
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    callbacks=callbacks
)

print("\n" + "="*70)
print("\n✅ Training complete!")
print(f"\n📊 Final Training Loss: {train_result.training_loss:.4f}")

## 💾 Step 7: Save Trained Model

Save the fine-tuned model for later use or download.

In [ ]:
print("💾 Saving trained model...\n")

# Save model (merged 16bit for best quality)
output_dir = "./models/raft_qwen3_colab"

trainer.save_model(
    output_dir=output_dir,
    save_method="merged_16bit"  # or "lora" for smaller size
)

print(f"\n✅ Model saved to {output_dir}")

# Check model size
import os
total_size = sum(os.path.getsize(os.path.join(dirpath, filename))
                 for dirpath, _, filenames in os.walk(output_dir)
                 for filename in filenames)
print(f"📦 Model size: {total_size / 1e9:.2f} GB")

# Optionally zip for download
print("\n📦 Creating zip file for download...")
!zip -r raft_model_colab.zip {output_dir}
print("✅ Model zipped as raft_model_colab.zip")
print("\nYou can download this file from the Files panel on the left.")

## 📊 Step 8: Evaluate Model

Test the fine-tuned model on evaluation examples.

In [ ]:
from raft_evaluator import RAFTEvaluator
from unsloth import FastLanguageModel

print("📊 Evaluating trained model...\n")

# Load model for inference
print("📦 Loading model for evaluation...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=output_dir,
    max_seq_length=config.model.max_seq_length,
    dtype=None,
    load_in_4bit=True,
)
FastLanguageModel.for_inference(model)

# Initialize evaluator
evaluator = RAFTEvaluator(
    config=config,
    retrieval_system=retrieval_system,
    model=model,
    tokenizer=tokenizer
)

# Prepare test examples (use dev set)
print("\n📝 Preparing evaluation examples...")
eval_examples = []
for ex in dev_prep[:5]:  # Evaluate on 5 examples
    eval_examples.append({
        'query': ex['query'],
        'gold_docs': [doc_id for doc_id, _ in ex['oracle_docs']],
        'answers': ex['answers']
    })

print(f"Evaluating on {len(eval_examples)} examples...\n")

# Run evaluation
metrics = evaluator.evaluate_dataset(
    eval_examples,
    output_path="./results/eval_colab.jsonl"
)

# Print results
print("\n" + "="*70)
evaluator.print_metrics(metrics)

# Show example generation
print("\n📝 Example Generation:")
print("="*70)
test_query = eval_examples[0]['query']
results = retrieval_system.retrieve(test_query, top_k=3)
contexts = [r.text for r in results]

print(f"Query: {test_query}\n")
generated = evaluator.generate_answer(test_query, contexts)
print(f"Generated Answer:\n{generated}\n")
print(f"Reference Answers:")
for i, ref in enumerate(eval_examples[0]['answers'], 1):
    print(f"  {i}. {ref[:100]}...")

print("\n✅ Evaluation complete!")

## 🎨 Step 9: Interactive Demo

Try the model with your own questions!

In [ ]:
def answer_question(question, top_k=3):
    """Answer a question using the trained model"""
    print(f"\n❓ Question: {question}")
    print("\n🔍 Retrieving relevant contexts...")
    
    # Retrieve contexts
    results = retrieval_system.retrieve(question, top_k=top_k)
    contexts = [r.text for r in results]
    
    print(f"   Found {len(results)} relevant documents")
    for i, r in enumerate(results, 1):
        print(f"   {i}. {r.doc_id} (score: {r.score:.4f})")
    
    print("\n🤖 Generating answer...\n")
    
    # Generate answer
    answer = evaluator.generate_answer(question, contexts)
    
    print("="*70)
    print("ANSWER:")
    print("="*70)
    print(answer)
    print("="*70)
    
    return answer

# Try some example questions
print("🎨 Interactive Demo - Try the model!\n")

# Example 1
answer_question("What is the main theme of the story?")

# Example 2
answer_question("Who are the main characters?")

# Try your own question
print("\n" + "="*70)
print("Try your own question!")
print("="*70)
custom_question = input("Enter your question: ")
if custom_question.strip():
    answer_question(custom_question)

## 📥 Step 10: Download Results

Package and download training artifacts.

In [ ]:
from google.colab import files
import shutil

print("📥 Preparing files for download...\n")

# Create results package
package_dir = "raft_training_results"
!mkdir -p {package_dir}

# Copy important files
print("📦 Packaging results...")

files_to_package = [
    ("colab_config.yaml", "Configuration"),
    ("./logs/training_colab.jsonl", "Training logs"),
    ("./results/eval_colab.jsonl", "Evaluation results"),
    ("./data/raft_train_colab.jsonl", "Training data sample"),
]

for file_path, description in files_to_package:
    if os.path.exists(file_path):
        shutil.copy(file_path, package_dir)
        print(f"  ✓ {description}")

# Create summary report
summary = f"""RAFT Training Summary
=====================

Training Configuration:
- Model: Qwen3-4B-Instruct
- LoRA Rank: {config.model.lora_r}
- Sequence Length: {config.model.max_seq_length}
- Batch Size: {config.training.per_device_train_batch_size}
- Gradient Accumulation: {config.training.gradient_accumulation_steps}
- Epochs: {config.training.num_train_epochs}
- Learning Rate: {config.training.learning_rate}

Dataset:
- Training Examples: {len(train_dataset)}
- Evaluation Examples: {len(eval_dataset) if eval_dataset else 0}
- Oracle Percentage: {config.raft_data.oracle_percentage}
- Distractors: {config.raft_data.num_distractors}

Results:
- Final Training Loss: {train_result.training_loss:.4f}
- Evaluation Metrics: See eval_colab.jsonl

Model Location: {output_dir}
Model Size: {total_size / 1e9:.2f} GB
"""

with open(f"{package_dir}/SUMMARY.txt", 'w') as f:
    f.write(summary)

print("  ✓ Summary report")

# Zip everything
print("\n📦 Creating zip file...")
!zip -r raft_results.zip {package_dir}

print("\n✅ Results packaged!")
print("\nDownload options:")
print("1. raft_results.zip - Training logs and results")
print("2. raft_model_colab.zip - Trained model (large file)")
print("\nUse the Files panel on the left to download.")

# Optionally trigger download
print("\n💾 Downloading results package...")
files.download('raft_results.zip')

## 📊 Step 11: Visualize Training Progress

Plot training metrics.

In [ ]:
import json
import matplotlib.pyplot as plt

print("📊 Visualizing training progress...\n")

# Load training logs
log_file = "./logs/training_colab.jsonl"
if os.path.exists(log_file):
    logs = []
    with open(log_file, 'r') as f:
        for line in f:
            logs.append(json.loads(line))
    
    # Extract metrics
    steps = [log['step'] for log in logs if 'loss' in log]
    losses = [log['loss'] for log in logs if 'loss' in log]
    learning_rates = [log.get('learning_rate', 0) for log in logs if 'loss' in log]
    
    # Create plots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Loss plot
    ax1.plot(steps, losses, 'b-', linewidth=2)
    ax1.set_xlabel('Training Steps', fontsize=12)
    ax1.set_ylabel('Loss', fontsize=12)
    ax1.set_title('Training Loss', fontsize=14, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    
    # Learning rate plot
    ax2.plot(steps, learning_rates, 'r-', linewidth=2)
    ax2.set_xlabel('Training Steps', fontsize=12)
    ax2.set_ylabel('Learning Rate', fontsize=12)
    ax2.set_title('Learning Rate Schedule', fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('training_progress.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("✅ Training visualization complete!")
    print(f"   Total steps: {len(steps)}")
    if losses:
        print(f"   Initial loss: {losses[0]:.4f}")
        print(f"   Final loss: {losses[-1]:.4f}")
        print(f"   Improvement: {((losses[0] - losses[-1]) / losses[0] * 100):.2f}%")
else:
    print("⚠️ Training log file not found")

## 🛠️ Step 12: Troubleshooting

Run diagnostics if you encounter issues.

In [ ]:
# Run this cell if you encounter issues

print("🔍 System Diagnostics\n")
print("="*70)

# Check GPU
import torch
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"GPU Memory Allocated: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB")
    print(f"GPU Memory Cached: {torch.cuda.memory_reserved(0) / 1e9:.2f} GB")

# Check disk space
import shutil
total, used, free = shutil.disk_usage("/")
print(f"\nDisk Space:")
print(f"  Total: {total / 1e9:.2f} GB")
print(f"  Used: {used / 1e9:.2f} GB")
print(f"  Free: {free / 1e9:.2f} GB")

# Check Python packages
print(f"\nKey Package Versions:")
import transformers
print(f"  transformers: {transformers.__version__}")
print(f"  torch: {torch.__version__}")

try:
    from unsloth import __version__ as unsloth_version
    print(f"  unsloth: {unsloth_version}")
except:
    print(f"  unsloth: installed (version unknown)")

# Check environment variables
print(f"\nEnvironment:")
print(f"  OPENAI_API_KEY: {'Set' if os.environ.get('OPENAI_API_KEY') else 'Not set'}")

print("\n" + "="*70)
print("\nCommon Solutions:")
print("  1. Out of Memory: Reduce batch_size or max_seq_length")
print("  2. API Errors: Check OpenAI API key and credits")
print("  3. Slow Training: Ensure GPU runtime is enabled")
print("  4. Import Errors: Restart runtime and reinstall packages")
print("="*70)

## 💾 Step 13: Save to Google Drive (Optional)

Save your work to Google Drive to prevent data loss.

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Create backup directory
backup_dir = '/content/drive/MyDrive/RAFT_Backup'
!mkdir -p {backup_dir}

print(f"📁 Backing up to: {backup_dir}\n")

# Copy important files
print("💾 Copying files...")
!cp -r ./models/raft_qwen3_colab {backup_dir}/ 2>/dev/null || echo "  ⚠️ Model not found"
!cp -r ./data {backup_dir}/ 2>/dev/null || echo "  ⚠️ Data not found"
!cp -r ./results {backup_dir}/ 2>/dev/null || echo "  ⚠️ Results not found"
!cp -r ./logs {backup_dir}/ 2>/dev/null || echo "  ⚠️ Logs not found"
!cp colab_config.yaml {backup_dir}/ 2>/dev/null || echo "  ⚠️ Config not found"

print("\n✅ Backup complete!")
print(f"Files saved to: {backup_dir}")

## 🎉 Congratulations!

You've successfully completed the RAFT fine-tuning pipeline!

### What you've accomplished:

✅ Installed all dependencies  
✅ Loaded MSRS Story-QA dataset  
✅ Built vector search index  
✅ Generated RAFT training data with CoT  
✅ Fine-tuned Qwen3-4B with QLoRA  
✅ Evaluated model performance  
✅ Created interactive demo  

### Next Steps:

1. **Increase dataset size** - Train on 100+ examples for better performance
2. **Tune hyperparameters** - Adjust learning rate, batch size, epochs
3. **Try different models** - Experiment with other base models
4. **Compare baselines** - Test against 0-shot and standard SFT
5. **Deploy the model** - Use for production QA tasks

### Resources:

- 📚 [RAFT Paper](https://arxiv.org/abs/2403.10131)
- 🔧 [GitHub Repository](https://github.com/limcheekin/MSRS-RAFT)
- 📖 [Unsloth Documentation](https://docs.unsloth.ai/)
- 💬 [MSRS Dataset](https://huggingface.co/datasets/yale-nlp/MSRS)

### Need Help?

- Check the [GitHub Issues](https://github.com/limcheekin/MSRS-RAFT/issues)
- Review the [COLAB_GUIDE.md](https://github.com/limcheekin/MSRS-RAFT/blob/main/COLAB_GUIDE.md)
- Run the troubleshooting cell above

---

**Happy Training! 🚀**